In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
from PIL import Image

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("wandb_api")
    wandb.login(key=api_key)
    anony = None
except:
    anony = "must"
    print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
df = pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
df.head()

In [ ]:
# Convert String to List
df['annotations'] = df['annotations'].apply(eval)

# Get the number of bounding boxes for each image
df['num_bboxes'] = df['annotations'].apply(lambda x: len(x))

In [ ]:
df.groupby('video_id').count()

In [ ]:
df_with_boxes = df[df.num_bboxes != 0].reset_index(drop=True)
df_with_boxes.shape

In [ ]:
run = wandb.init(project='GreatBarrier',
                 job_type='Visualization',
                 name='Image Visualization',
                 anonymous='must')

In [ ]:
preview_table = wandb.Table(columns=['Video Id', 'Image', 'Video Frame', 'Num Boxes'])

for i in tqdm(range(len(df_with_boxes))):
    row = df_with_boxes.loc[i]
    img = Image.open(f'../input/tensorflow-great-barrier-reef/train_images/video_{row.video_id}/{row.video_frame}.jpg')
    bboxes_list = []
    for annot in row.annotations:
        bbox = {
            "position": {
                "minX": annot['x'],
                "maxX": annot['x'] + annot['width'],
                "minY": annot['y'],
                 "maxY": annot['y'] + annot['height']
            },
            "class_id": 1,
            "box_caption": "starfish",
            "domain": "pixel"
        }
        bboxes_list.append(bbox)
        
    image = wandb.Image(img,
                        boxes = {
                            "ground_truth": {
                                "box_data": bboxes_list,
                                "class_labels" : {1: 'starfish'}
                                 }
                        },
                        # Add extra dummy class to get red bounding boxes
                        classes = [{"id": 0, "name": "none"}, {"id": 1, "name": "starfish"}]
                    )
    preview_table.add_data(row.video_id,
                           image,
                           row.video_frame,
                           len(bboxes_list))

wandb.log({'Visualization': preview_table})
run.finish()